In [1]:
import pandas as pd
db=pd.read_csv('adult.csv')

In [2]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [3]:
db['occupation'].value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
?                    2809
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [4]:
res=[]
for i in db.columns:
    if '?' in db[i].value_counts().index.tolist():
        res.append(db[i].value_counts()['?'])
    else:
        res.append(0)

In [5]:
res

[0, 2799, 0, 0, 0, 0, 2809, 0, 0, 0, 0, 0, 0, 857, 0]

In [6]:
db[db['occupation']=='?']['workclass'].value_counts()

?               2799
Never-worked      10
Name: workclass, dtype: int64

In [7]:
db[db['workclass']=='?']['occupation'].value_counts()

?    2799
Name: occupation, dtype: int64

примем решение удалить строки со знаками вопроса, их не так много

In [8]:
db.loc[6]

age                           29
workclass                      ?
fnlwgt                    227026
education                HS-grad
educational-num                9
marital-status     Never-married
occupation                     ?
relationship           Unmarried
race                       Black
gender                      Male
capital-gain                   0
capital-loss                   0
hours-per-week                40
native-country     United-States
income                     <=50K
Name: 6, dtype: object

In [9]:
for i in range(48842):
    if '?' in db.loc[i].values.tolist():
        db=db.drop(i, axis=0)

In [10]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
age                45222 non-null int64
workclass          45222 non-null object
fnlwgt             45222 non-null int64
education          45222 non-null object
educational-num    45222 non-null int64
marital-status     45222 non-null object
occupation         45222 non-null object
relationship       45222 non-null object
race               45222 non-null object
gender             45222 non-null object
capital-gain       45222 non-null int64
capital-loss       45222 non-null int64
hours-per-week     45222 non-null int64
native-country     45222 non-null object
income             45222 non-null object
dtypes: int64(6), object(9)
memory usage: 6.8+ MB


In [11]:
db['income'].value_counts()

<=50K    34014
>50K     11208
Name: income, dtype: int64

кодируем таргет, как 1 и 0

In [12]:
income_res=[]
for i in db['income'].values.tolist():
    if i=='<=50K':
        income_res.append(1)
    else:
        income_res.append(0)
db['income_target']=income_res
db=db.drop('income', axis=1)

делим на тест и трейн

In [13]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(db, test_size=0.2, random_state=42)

In [14]:
import numpy as np
res2=[]
for i in db.columns:
    res1=[]
    for j in db[i].values.tolist():
        if isinstance(j, str)==True:
            res1.append(1)
        else:
            res1.append(0)
    if sum(res1)>0:
        res2.append(i)
    else:
        res2.append(0)
print (res2)

[0, 'workclass', 0, 'education', 0, 'marital-status', 'occupation', 'relationship', 'race', 'gender', 0, 0, 0, 'native-country', 0]


In [15]:
category_columns=[x for x in res2 if x!=0]

кодируем категориальные признаки

In [16]:
def get_woe_v1(db, db_test, col, target_col):
    all_good = len(db[db[target_col] == 1][col])
    all_bad = len(db[db[target_col] == 0][col])
    odds_series = (
        db[db[target_col] == 1][col].value_counts()
        /
        db[db[target_col] == 0][col].value_counts()
    )
    odds_series = odds_series / all_good * all_bad
    category_woe_dict = np.log(odds_series).to_dict()
    db[col + '_woe'] = db[col].apply(category_woe_dict.get)
    db_test[col + '_woe'] = db_test[col].apply(category_woe_dict.get)
    return db, db_test

In [17]:
for i in category_columns:
    get_woe_v1(train_set, test_set ,i, 'income_target')

C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\volko\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [18]:
train_set=train_set.drop(category_columns, axis=1)
test_set=test_set.drop(category_columns, axis=1)

In [19]:
corr_matrix=train_set.corr()

In [20]:
corr_matrix

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income_target,workclass_woe,education_woe,marital-status_woe,occupation_woe,relationship_woe,race_woe,gender_woe,native-country_woe
age,1.000000,-0.074184,0.039578,0.079746,0.059814,0.103438,-0.239593,-0.184865,-0.086794,-0.385446,-0.144186,-0.408481,-0.026356,-0.081418,-0.060128
fnlwgt,-0.074184,1.000000,-0.041268,-0.000201,-0.006288,-0.021149,0.005027,0.037004,0.037328,0.034522,0.010689,0.032467,0.103301,-0.027418,0.120438
educational-num,0.039578,-0.041268,1.000000,0.127142,0.081148,0.149340,-0.333502,-0.150889,-0.966184,-0.073788,-0.454160,-0.122541,-0.092575,-0.005935,-0.253661
capital-gain,0.079746,-0.000201,0.127142,1.000000,-0.032315,0.083739,-0.221154,-0.090947,-0.148052,-0.081119,-0.098178,-0.083861,-0.026603,-0.051509,-0.018453
capital-loss,0.059814,-0.006288,0.081148,-0.032315,1.000000,0.051973,-0.148969,-0.043774,-0.089599,-0.074451,-0.081657,-0.078864,-0.024563,-0.044163,-0.022906
hours-per-week,0.103438,-0.021149,0.149340,0.083739,0.051973,1.000000,-0.230053,-0.129021,-0.170677,-0.240760,-0.215435,-0.289865,-0.049973,-0.232458,-0.019033
income_target,-0.239593,0.005027,-0.333502,-0.221154,-0.148969,-0.230053,1.000000,0.165910,0.352802,0.444984,0.335405,0.435619,0.097773,0.217528,0.095822
workclass_woe,-0.184865,0.037004,-0.150889,-0.090947,-0.043774,-0.129021,0.165910,1.000000,0.163468,0.143794,0.181977,0.146810,0.021860,0.087234,0.054968
education_woe,-0.086794,0.037328,-0.966184,-0.148052,-0.089599,-0.170677,0.352802,0.163468,1.000000,0.100751,0.463618,0.154485,0.098757,0.021109,0.204705
marital-status_woe,-0.385446,0.034522,-0.073788,-0.081119,-0.074451,-0.240760,0.444984,0.143794,0.100751,1.000000,0.214432,0.892875,0.123564,0.400468,0.030330


In [21]:
train_set=train_set.drop('educational-num', axis=1)
test_set=test_set.drop('educational-num', axis=1)

удалили коррелирующие факторы, мультколлинеарность не сильно выражена

In [22]:
corr_target=corr_matrix['income_target'].abs().sort_values()

In [23]:
corr_target_list=corr_target.index.tolist()

In [24]:
corr_target_list

['fnlwgt',
 'native-country_woe',
 'race_woe',
 'capital-loss',
 'workclass_woe',
 'gender_woe',
 'capital-gain',
 'hours-per-week',
 'age',
 'educational-num',
 'occupation_woe',
 'education_woe',
 'relationship_woe',
 'marital-status_woe',
 'income_target']

In [25]:
corr_target_list.remove('marital-status_woe')

In [26]:
corr_target_list.remove('educational-num')

In [27]:
corr_target_list.remove('income_target')

In [28]:
train_set=train_set.dropna()

In [29]:
X=train_set.drop('income_target', axis=1)
Y=train_set['income_target']

In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36161 entries, 8605 to 17081
Data columns (total 13 columns):
age                   36161 non-null int64
fnlwgt                36161 non-null int64
capital-gain          36161 non-null int64
capital-loss          36161 non-null int64
hours-per-week        36161 non-null int64
workclass_woe         36161 non-null float64
education_woe         36161 non-null float64
marital-status_woe    36161 non-null float64
occupation_woe        36161 non-null float64
relationship_woe      36161 non-null float64
race_woe              36161 non-null float64
gender_woe            36161 non-null float64
native-country_woe    36161 non-null float64
dtypes: float64(8), int64(5)
memory usage: 3.9 MB


перебираем признаки

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X, Y)
y_predict=model.predict(X)
model.score(X, Y)
roc_auc1 = np.round(roc_auc_score(Y, y_predict), 2)
roc_auc_metr=[]
roc_auc_metr.append(roc_auc1)
for i in corr_target_list:
    X=X.drop(i, axis=1)
    model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X, Y)
    y_predict=model.predict(X)
    roc_auc = np.round(roc_auc_score(Y, y_predict), 2)
    roc_auc_metr.append(roc_auc)

In [32]:
roc_auc_metr

[0.62, 0.75, 0.75, 0.76, 0.75, 0.74, 0.76, 0.73, 0.72, 0.72, 0.68, 0.5, 0.5]

In [37]:
X=train_set.drop(['income_target', 'fnlwgt','native-country_woe','race_woe'], axis=1)
model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X, Y)
y_predict=model.predict(X)
score=model.score(X, Y)
roc_auc = np.round(roc_auc_score(Y, y_predict), 2)

In [38]:
print (score, roc_auc)

0.8383617709687232 0.76


In [39]:
X_test=test_set.drop(['income_target', 'fnlwgt','native-country_woe','race_woe'], axis=1)
Y_test=test_set['income_target']
model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_test, Y_test)
y_predict_test=model.predict(X_test)
score_test=model.score(X_test, Y_test)
roc_auc_test = np.round(roc_auc_score(Y_test, y_predict_test), 2)

In [40]:
print (score_test, roc_auc_test)

0.8441127694859039 0.75
